## Homework - Unsupervised

Kelompok 2:

*   Oei, Gabrielle Maureen B.S
*   Ummi Sarah
*   Arini Arumsari
*   Althaaf Athaayaa Daffa Qushayyizidane
*   Ruslan Hadi
*   Muhammad Ikhsan Hendriyana
*   Taufik Ismail
*   Jackson
*   Tsany Sutia

In [1]:
# Melakukan import library
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

import gdown

print('numpy version : ',np.__version__)
print('pandas version : ',pd.__version__)
print('seaborn version : ',sns.__version__)

ModuleNotFoundError: No module named 'gdown'

In [21]:
sns.set(rc={'figure.figsize':(20.7,8.27)})
sns.set_style("whitegrid")
sns.color_palette("dark")
plt.style.use("fivethirtyeight")

In [22]:
# Melakukan import csv
df = pd.read_csv("flight.csv")
df.sample(5)

,MEMBER_NO,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,FFP_TIER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,AGE,LOAD_TIME,...,SUM_YR_2,SEG_KM_SUM,LAST_FLIGHT_DATE,LAST_TO_END,AVG_INTERVAL,MAX_INTERVAL,EXCHANGE_COUNT,avg_discount,Points_Sum,Point_NotFlight
19748,49973,12/10/2010,12/10/2010,Male,4,dongguan,guangdong,CN,33.0,3/31/2014,...,5201.0,16469,2014/2/29 0:00:00,32,56.818182,144,0,0.751251,11083,0
863,50367,1/6/2012,2/9/2012,Male,5,dalian,liaoning,CN,54.0,3/31/2014,...,30713.0,90418,3/29/2014,3,7.402062,117,0,0.829356,70498,4
17102,50959,1/25/2012,2/17/2012,Male,4,zhaoyangqu,beijingshi,CN,35.0,3/31/2014,...,8487.0,17040,3/30/2014,2,55.923077,244,0,0.835457,13322,0
2811,16774,6/16/2009,6/18/2009,Male,4,beijing,beijing,CN,46.0,3/31/2014,...,12668.0,48998,2/2/2014,59,23.285714,97,3,0.922505,47845,4
58822,49483,4/28/2010,8/16/2010,Male,4,kunming,yunnan,CN,58.0,3/31/2014,...,635.0,2181,1/24/2014,68,114.000000,292,0,0.665525,1251,0


# Describe Data

### Missing Value

In [51]:
# Melihat jumlah baris dan kolom
test = df.shape[0]

# Mengecek missing value ditiap fitur
missing_values_count = df.isnull().sum()
missing_values_count

MEMBER_NO               0
FFP_DATE                0
FIRST_FLIGHT_DATE       0
GENDER                  3
FFP_TIER                0
WORK_CITY            2269
WORK_PROVINCE        3248
WORK_COUNTRY           26
AGE                     0
LOAD_TIME               0
FLIGHT_COUNT            0
BP_SUM                  0
SUM_YR_1              551
SUM_YR_2              138
SEG_KM_SUM              0
LAST_FLIGHT_DATE        0
LAST_TO_END             0
AVG_INTERVAL            0
MAX_INTERVAL            0
EXCHANGE_COUNT          0
avg_discount            0
Points_Sum              0
Point_NotFlight         0
dtype: int64

In [42]:
# Mengecek datatype dari masing-masing kolom
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62988 entries, 0 to 62987
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MEMBER_NO          62988 non-null  int64  
 1   FFP_DATE           62988 non-null  object 
 2   FIRST_FLIGHT_DATE  62988 non-null  object 
 3   GENDER             62985 non-null  object 
 4   FFP_TIER           62988 non-null  int64  
 5   WORK_CITY          60719 non-null  object 
 6   WORK_PROVINCE      59740 non-null  object 
 7   WORK_COUNTRY       62962 non-null  object 
 8   AGE                62988 non-null  float64
 9   LOAD_TIME          62988 non-null  object 
 10  FLIGHT_COUNT       62988 non-null  int64  
 11  BP_SUM             62988 non-null  int64  
 12  SUM_YR_1           62437 non-null  float64
 13  SUM_YR_2           62850 non-null  float64
 14  SEG_KM_SUM         62988 non-null  int64  
 15  LAST_FLIGHT_DATE   62988 non-null  object 
 16  LAST_TO_END        629

### Masih ada data yang kosong Tolong di koreksi

In [ ]:
# # Mengisi nilai kosong di kolom 'GENDER' dengan nilai default, misalnya 'Unknown'
# df['GENDER'].fillna('Unknown', inplace=True)

# Duplicate Data

In [60]:
df.duplicated().sum()

0

In [61]:
df.duplicated(subset=['MEMBER_NO'],keep=False).sum()

0

In [35]:
# Pengelompokan kolom berdasarkan jenisnya
num = ['int64', 'int32', 'int16', 'float64', 'float32', 'float16']
num = df.select_dtypes(include=num)
num.drop(columns=['MEMBER_NO'], inplace=True)
num = num.columns
cats = ['object', 'bool']
cats = df.select_dtypes(include=cats)
cats = cats.columns


In [36]:
df[num].describe()

,FFP_TIER,AGE,FLIGHT_COUNT,BP_SUM,SUM_YR_1,SUM_YR_2,SEG_KM_SUM,LAST_TO_END,AVG_INTERVAL,MAX_INTERVAL,EXCHANGE_COUNT,avg_discount,Points_Sum,Point_NotFlight
count,62988.000000,62568.000000,62988.000000,62988.000000,62437.000000,62850.000000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000,62988.000000,62988.0000,62988.000000
mean,4.102162,42.476346,11.839414,10925.081254,5355.376064,5604.026014,17123.878691,176.120102,67.749788,166.033895,0.319775,0.721558,12545.7771,2.728155
std,0.373856,9.885915,14.049471,16339.486151,8109.450147,8703.364247,20960.844623,183.822223,77.517866,123.397180,1.136004,0.185427,20507.8167,7.364164
min,4.000000,6.000000,2.000000,0.000000,0.000000,0.000000,368.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
25%,4.000000,35.000000,3.000000,2518.000000,1003.000000,780.000000,4747.000000,29.000000,23.370370,79.000000,0.000000,0.611997,2775.0000,0.000000
50%,4.000000,41.000000,7.000000,5700.000000,2800.000000,2773.000000,9994.000000,108.000000,44.666667,143.000000,0.000000,0.711856,6328.5000,0.000000
75%,4.000000,48.000000,15.000000,12831.000000,6574.000000,6845.750000,21271.250000,268.000000,82.000000,228.000000,0.000000,0.809476,14302.5000,1.000000
max,6.000000,110.000000,213.000000,505308.000000,239560.000000,234188.000000,580717.000000,731.000000,728.000000,728.000000,46.000000,1.500000,985572.0000,140.000000


In [37]:
df[cats].describe()

,FFP_DATE,FIRST_FLIGHT_DATE,GENDER,WORK_CITY,WORK_PROVINCE,WORK_COUNTRY,LOAD_TIME,LAST_FLIGHT_DATE
count,62988,62988,62985,60719,59740,62962,62988,62988
unique,3068,3406,2,3234,1165,118,1,731
top,1/13/2011,2/16/2013,Male,guangzhou,guangdong,CN,3/31/2014,3/31/2014
freq,184,96,48134,9386,17509,57748,62988,959


In [72]:
# Mengecek unique value, modus, dan frekuensi modus dari tiap kolom
for i in df:
  x = df[i].mode()[0]
  y = df[i].value_counts()[x]
  z = df[i].nunique()
  print(f'Terdapat {z} unique values dan modus dari kolom {i} = {x} dengan frekuensi sebesar {y}')

Terdapat 62988 unique values dan modus dari kolom MEMBER_NO = 1 dengan frekuensi sebesar 1
Terdapat 3068 unique values dan modus dari kolom FFP_DATE = 1/13/2011 dengan frekuensi sebesar 184
Terdapat 3406 unique values dan modus dari kolom FIRST_FLIGHT_DATE = 2/16/2013 dengan frekuensi sebesar 96
Terdapat 3 unique values dan modus dari kolom GENDER = Male dengan frekuensi sebesar 48134
Terdapat 3 unique values dan modus dari kolom FFP_TIER = 4 dengan frekuensi sebesar 58066
Terdapat 3234 unique values dan modus dari kolom WORK_CITY = guangzhou dengan frekuensi sebesar 9386
Terdapat 1165 unique values dan modus dari kolom WORK_PROVINCE = guangdong dengan frekuensi sebesar 17509
Terdapat 118 unique values dan modus dari kolom WORK_COUNTRY = CN dengan frekuensi sebesar 57748
Terdapat 85 unique values dan modus dari kolom AGE = 47.0 dengan frekuensi sebesar 2727
Terdapat 1 unique values dan modus dari kolom LOAD_TIME = 3/31/2014 dengan frekuensi sebesar 62988
Terdapat 153 unique values dan 

In [39]:
# Mengecek Pesebaran Data
nums = df[num]
for i in nums:
  minv = nums[i].min()
  maxv = nums[i].max()
  Q1 = nums[i].quantile(.25)
  Q3 = nums[i].quantile(.75)
  total_range = maxv-minv
  var = round(nums[i].var(),2)
  iqr = Q3-Q1
  print (f'Ukuran penyebaran dari kolom {i}, variance = {var}, range = {total_range}, IQR = {iqr}')

Ukuran penyebaran dari kolom FFP_TIER, variance = 0.14, range = 2, IQR = 0.0
Ukuran penyebaran dari kolom AGE, variance = 97.73, range = 104.0, IQR = 13.0
Ukuran penyebaran dari kolom FLIGHT_COUNT, variance = 197.39, range = 211, IQR = 12.0
Ukuran penyebaran dari kolom BP_SUM, variance = 266978807.69, range = 505308, IQR = 10313.0
Ukuran penyebaran dari kolom SUM_YR_1, variance = 65763181.68, range = 239560.0, IQR = 5571.0
Ukuran penyebaran dari kolom SUM_YR_2, variance = 75748549.22, range = 234188.0, IQR = 6065.75
Ukuran penyebaran dari kolom SEG_KM_SUM, variance = 439357007.32, range = 580349, IQR = 16524.25
Ukuran penyebaran dari kolom LAST_TO_END, variance = 33790.61, range = 730, IQR = 239.0
Ukuran penyebaran dari kolom AVG_INTERVAL, variance = 6009.02, range = 728.0, IQR = 58.62962963
Ukuran penyebaran dari kolom MAX_INTERVAL, variance = 15226.86, range = 728, IQR = 149.0
Ukuran penyebaran dari kolom EXCHANGE_COUNT, variance = 1.29, range = 46, IQR = 0.0
Ukuran penyebaran dari k

# Univariate Analysis